<h4>Lily Grumbach
    <br>M1 Humanités numériques - Université PSL</h4>
<h1><center>Rendu TAIS-TAL </center></h1>
<h2><center>Partie TAIS</center></h2>
<h3><center>4/5 : Reconstituer les coordonnées géographiques des GPH et de l'IREL</center></h3>

<h3><u>PLAN:</u></h3>

1) Harmonisation des données Géopolitiques pour le fond de carte de 1914
<br>
    
2) Explorer les bases de données à ma disposition (IREL et GPH) et les préparer pour l'extraction des coordonnées géographiques 
<br/>
 
3) Reconstituer les liens URL de la base de données IREL des Archives nationales d'Outre-mer.

<h4>4) Reconstituer les coordonnées géographiques des GPH et de l'IREL</h4>
 
5) Géoréférencer les données de mes revues


<b>Stratégie en place :  </b>

Nous avons eu recours à trois sources de coordonnées géographiques: 
* **Wikidata** indiqués par le medialab 
* l'**IREL**, la base de données géographique de l'inventaire en ligne des Archives nationales d'Outre Mer
* **Geonames**, pour le meilleur et pour le pire.

<h3>Objectifs de la partie : </h3>

Récupérer les coordonnées décimales d'entités géographiques historiques pour pouvoir l'appliquer sans problème à l'avenir.

In [1]:
#module
import desambiguisation

##General : 
from desambiguisation import ListEntities2df
##GPH
from desambiguisation import MatchGPELOC_IREL,Extract_longlat_WikiData
##IREL : 
from desambiguisation import nettoyage_df_IREL,urlencode,Extract_longlat_IREL,IREL_Nettoyage_AdminLieuDit,nettoyage_desambiguisation

#Généraux
import pandas as pd
import numpy as np
import re

#Webscrapping
from bs4 import BeautifulSoup
import geocoder
from lxml import html
import requests

#Wikidata 
import qwikidata
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api

# 0. Import des données : 

In [2]:
#Liste des entités GPELOC à annoter :
df_GPELOC=pd.read_csv("./output/2-Exploration_GPH-IREL/GPELOC_à_annoter.csv")

#df des lieux en 1914 avec leur lien Wikidata :
df_WikiQGIS = pd.read_csv("./output/2-Exploration_GPH-IREL/WikiQGIS.csv")

#df de l'ensemble des lieux de GPH avec leur lien wikidata
df_wikiData = pd.read_csv("./GeoPolHist-202103/medialab-GeoPolHist-fb19b66/data/GeoPolHist_entities.csv")
df_wikiData=df_wikiData.rename(columns={"GPH_name":"NAME"})

#df des lieux dans l'IREL avec leurs coordonnées géographiques


In [3]:
df_GPELOC

,NAME
0,Acores
1,Algérie
2,Allemagne
3,Angleterre
4,Annam
...,...
220,lac Tchad.
221,île de Saint-Barthélémy
222,île de Saint-Martin
223,île de la Réunion


In [4]:
dico_IREL ={}
n=0

with open('./IREL/liste_lieux_IREL.js','r') as f:
    lignes = f.readlines()
    
    for l in lignes:
        l=l.split("\n")
        dico_IREL[n]=l
        n+=1

df_IREL = nettoyage_df_IREL(dico_IREL)

return: df_IREL nettoyé


In [5]:
df_IREL=df_IREL.rename(columns={"Lieu-dit":"NAME"})
# df_IREL

# 1. GPH : Retrouver dans wikidata les coordonnées géographiques

=> La méthode que nous avons mis en place pour récupérer l'ensemble des coordonnées n'a pas été concluante malgré les nombreux efforts pour éviter l'ensemble des messages d'erreur provenantde WikiData. Nous envisageons reprendre la fonction du module pour l'appliquer par silots de 100 lieux afin d'avoir plus de contrôle dessus.

#### Pour extraire TOUTES les coordonnées géographiques des entités de `GPH`

In [12]:
pattern = r"(Q\d+)"
df_wikiData["Q_code"] = df_wikiData["wikidata"].str.extract(pattern)
df_wikiData.to_csv("./output/4-Reconstitution_CoordGéo_GPH-IREL/GPH_Wikidata_Qcode.csv")

List_Q_code_all =df_wikiData["Q_code"].tolist()
output_path_all = "./output/4-Reconstitution_CoordGéo_GPH-IREL/GPELOC_GPH_coordonnees_all.csv"

## ETAPE TRES LONGUE. Nous avons enregistré le resultat dans output_path_all 
# Extract_longlat_WikiData(List_Q_code_all,output_path_all)


In [13]:
#Importation des donnéesavec Q_code, latitude et longitude 
df_WikiData_longlat_all = pd.read_csv(output_path_all)

df_wikiData

,GPH_code,NAME,continent,wikidata,wikidata_alt1,wikidata_alt2,wikidata_alt3,notes,Q_code
0,2,United States of America,America,http://www.wikidata.org/entity/Q30,NaN,NaN,NaN,NaN,Q30
1,3,Alaska,America,http://www.wikidata.org/entity/Q797,NaN,NaN,NaN,NaN,Q797
2,4,Hawaii,America,http://www.wikidata.org/entity/Q782,NaN,NaN,NaN,NaN,Q782
3,5,Virgin Islands,America,http://www.wikidata.org/entity/Q11703,NaN,NaN,NaN,NaN,Q11703
4,6,Puerto Rico,America,http://www.wikidata.org/entity/Q1183,NaN,NaN,NaN,NaN,Q1183
...,...,...,...,...,...,...,...,...,...
1223,9994,Ross Dependency,Oceania,http://www.wikidata.org/entity/Q203073,NaN,NaN,NaN,NaN,Q203073
1224,9998,Concessions in China,World,https://www.wikidata.org/wiki/Q1376651,NaN,NaN,NaN,NaN,Q1376651
1225,9999,Inter-Allied Commission of Control,World,https://www.wikidata.org/wiki/Q1665892,NaN,NaN,NaN,NaN,Q1665892
1226,0,League of Nations,World,https://www.wikidata.org/wiki/Q38130,NaN,NaN,NaN,NaN,Q38130


Merge du df contenant les long/lat avec le df contenant les Noms et GPH_code de chaque entité

In [14]:
df_WikiData_longlat_Name = pd.merge(df_wikiData[["GPH_code","NAME","Q_code"]],df_WikiData_longlat_all,how="inner",on="Q_code")

Faire une estimation du nombre de Q_code n'ayant pas de coordonnées dans dans WikiData : 

In [15]:
print("Nombre de coordonnées manquantes dans WikiData : ",len(df_WikiData_longlat_Name.loc[df_WikiData_longlat_Name["latitude"]==0]))

Nombre de coordonnées manquantes dans WikiData :  5059


#### Récupérer les long/lat des données  `GPELOC` de 1898 à 1908 et les merge avec les longitudes et latitudes

La majeur différence entre la liste GPELOC à notre disposition et la base de donnée de GPH est la langue. GPH est en anglais tandis que GPELOC en français. On va donc ici d'abord "traduire" les entités nommées de la liste GPELOC en anglais.

N'ayant pas réussi à faire fonctionner `googletrans ` nous avons traduit directement sur Deepl en ligne

In [16]:
with open ("./desambiguisation/list_GPELOC_anglais.txt") as f:
    list_GPELOC_anglais = (f.readlines())

for i in range (len(list_GPELOC_anglais)):
    list_GPELOC_anglais[i]=list_GPELOC_anglais[i].replace(",\n","")
list_GPELOC_anglais

df_GPELOC_anglais=pd.DataFrame({"NAME":list_GPELOC_anglais})

In [17]:
##Faire un dictionnaire:
list_GPELOC=df_GPELOC.NAME.tolist()
dico_placeName = {}
for i in range(len(list_GPELOC_anglais)):
    placeNameEnglish = list_GPELOC_anglais[i]
    placeNameFrançais = list_GPELOC[i]
    dico_placeName[placeNameEnglish] = placeNameFrançais  
# dico_placeName

EN ANGLAIS : Création d'un df des entités de GPELOC trouvées dans WikiData_longlat

In [18]:
df_WikiData_longlat_18981914_anglais =pd.merge(df_WikiData_longlat_Name,df_GPELOC_anglais,how="inner",on="NAME")

#On remarque qu'il y a un doublon : 
df_WikiData_longlat_18981914_anglais=df_WikiData_longlat_18981914_anglais.drop_duplicates(keep="first")

df_WikiData_longlat_18981914_anglais.to_csv("./output/4-Reconstitution_CoordGéo_GPH-IREL/GPH_GPELOC_longlat_anglais.csv",index=False)

df_WikiData_longlat_18981914_anglais

,GPH_code,NAME,Q_code,latitude,longitude
0,40,Cuba,Q241,22.000000,-79.500000
1,53,Barbados,Q244,13.170000,-59.552500
2,65,Guadeloupe,Q17012,16.250000,-61.583333
3,66,Martinique,Q17054,14.666667,-61.000000
4,70,Mexico,Q96,23.000000,-102.000000
5,220,France,Q142,47.000000,2.000000
6,231,Gibraltar,Q1410,36.137750,-5.345361
7,255,Germany,Q183,51.000000,10.000000
8,350,Greece,Q41,38.500000,23.000000
9,351,Crete,Q34374,35.309722,24.893333


TRADUCTION du nom des lieux en français pour pouvoir en suite merge avec DF_GPELOC : 

In [19]:
df_WikiData_longlat_18981914=df_WikiData_longlat_18981914_anglais

#Méthode barbare, jesais:
liste_placeName_geoloc=[]
for placeNameEnglish in  df_WikiData_longlat_18981914_anglais["NAME"]:
    placeNameFrançais = dico_placeName[placeNameEnglish]
    liste_placeName_geoloc.append(placeNameFrançais)

df_WikiData_longlat_18981914["NAME"]=liste_placeName_geoloc
df_WikiData_longlat_18981914

,GPH_code,NAME,Q_code,latitude,longitude
0,40,Cuba,Q241,22.000000,-79.500000
1,53,Barbade,Q244,13.170000,-59.552500
2,65,Guadeloupe,Q17012,16.250000,-61.583333
3,66,Martinique,Q17054,14.666667,-61.000000
4,70,Mexique,Q96,23.000000,-102.000000
5,220,France,Q142,47.000000,2.000000
6,231,Gibraltar,Q1410,36.137750,-5.345361
7,255,Allemagne,Q183,51.000000,10.000000
8,350,Grèce,Q41,38.500000,23.000000
9,351,Crète,Q34374,35.309722,24.893333


#### Cette démarche a été concluante puisque l'on a récupéré 40 entités au lieu des 23 déclarées dans la partie 2 de ce rendu !

### On merge la liste des latitudes/longitudes `GPELOC_GPH ` (càd trouvées dans la base de données GPH) avec la liste de l'ensemble  des lieux à géoréférencer pour nos revues, `df_GPELOC`

In [22]:
# list_placesName_18981914 = df_WikiData_longlat_18981914.NAME.tolist()
# for placeName in list_placesName_18981914 :
#     list_GPELOC.remove(placeName)

On a alors la liste ` list_GPELOC ` qui ne contient que les noms de lieux dont il reste à trouver leslong/lat

# 2. IREL : Retrouver par webscrapping les coordonnées géographiques des Archives d'Outre-mer

Beaucoup plus compliqué.....

In [23]:
df_IREL

,Administration,NAME,Type,titre
0,None,None,None,var geographie = [Aachen (Allemagne)
1,République du Congo,Abala,Subdivision,AbalaSubdivision (République du Congo)
2,Bénin,Abananquem,None,Abananquem (Bénin)
3,Algérie,Abbaziz,None,Abbaziz (Algérie)
4,Somme ; France,Abbeville,None,Abbeville (Somme ; France)
...,...,...,...,...
14566,Allemagne,Zusenhofen,None,Zusenhofen (Allemagne)
14567,Nord ; France,Zuytpeene,None,Zuytpeene (Nord ; France)
14568,Algérie,Z\'Mala,None,Z\Mala (Algérie)
14569,Algérie,Z’Malet El Emir Abdelkader,None,Z’Malet El Emir Abdelkader (Algérie)


## 2.1. Chercher quels noms de lieux présents dans ma revue se trouve dans celle des IREL  

#### Import du df de ce que l'on peut et ne peut pas trouver avec l'IREL

In [24]:
comparaison_matchIREL=pd.read_csv("./desambiguisation/comparaison_match_IREL.csv")
comparaison_matchIREL

,L_MatchDF_IREL,L_NotMatchDF_IREL
0,Algérie,Acores
1,Allemagne,Angleterre
2,Bizerte,Annam
3,Bordeaux,Asie
4,Brest,Buenos-Ayres
...,...,...
108,Vinh,lac Tchad.
109,Wallis,île de Saint-Barthélémy
110,Yunnan,île de Saint-Martin
111,Zinder,île de la Réunion


#### Import du df IREL avec les coordonnées

In [26]:
df_IREL_url = pd.read_csv("./output/3-Reconstitution_URL_IREL/IREL_ens_url.csv")
df_IREL_url

,Unnamed: 0,Administration,NAME,Type,titre,titre_encoded,IREL_url,Coordonnees,latitude,longitude,Administration_bis,Lieu-dit_bis
0,0,NaN,NaN,NaN,var geographie = [Aachen (Allemagne),var%20geographie%20%3D%20%5BAachen%20%28Allema...,http://anom.archivesnationales.culture.gouv.fr...,NaN,NaN,NaN,NaN,NaN
1,1,République du Congo,Abala,Subdivision,AbalaSubdivision (République du Congo),AbalaSubdivision%20%28R%C3%A9publique%20du%20C...,http://anom.archivesnationales.culture.gouv.fr...,NaN,NaN,NaN,NaN,NaN
2,2,Bénin,Abananquem,NaN,Abananquem (Bénin),Abananquem%20%28B%C3%A9nin%29,http://anom.archivesnationales.culture.gouv.fr...,6.2850 1.8118,6.2850,1.8118,NaN,NaN
3,3,Algérie,Abbaziz,NaN,Abbaziz (Algérie),Abbaziz%20%28Alg%C3%A9rie%29,http://anom.archivesnationales.culture.gouv.fr...,34.6167 2.8000,34.6167,2.8000,NaN,NaN
4,4,Somme ; France,Abbeville,NaN,Abbeville (Somme ; France),Abbeville%20%28Somme%20%3B%20France%29,http://anom.archivesnationales.culture.gouv.fr...,50.1 1.8333,50.1000,1.8333,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14566,14566,Allemagne,Zusenhofen,NaN,Zusenhofen (Allemagne),Zusenhofen%20%28Allemagne%29,http://anom.archivesnationales.culture.gouv.fr...,48.5500 8.0167,48.5500,8.0167,NaN,NaN
14567,14567,Nord ; France,Zuytpeene,NaN,Zuytpeene (Nord ; France),Zuytpeene%20%28Nord%20%3B%20France%29,http://anom.archivesnationales.culture.gouv.fr...,50.7947 2.4275,50.7947,2.4275,NaN,NaN
14568,14568,Algérie,Z\'Mala,NaN,Z\Mala (Algérie),Z%5CMala%20%28Alg%C3%A9rie%29,http://anom.archivesnationales.culture.gouv.fr...,NaN,NaN,NaN,NaN,NaN
14569,14569,Algérie,Z’Malet El Emir Abdelkader,NaN,Z’Malet El Emir Abdelkader (Algérie),Z%E2%80%99Malet%20El%20Emir%20Abdelkader%20%28...,http://anom.archivesnationales.culture.gouv.fr...,34.9000 2.3500,34.9000,2.3500,NaN,NaN


#### Maintenant on cherche à extraire de ` df_IREL ` les coordonnées des lieux qui ont matché 

Le principal problème est que pour chaque entité qui "match", on peut avoir de nombreux résultats.Par exemple, "Algérie" compte plus 

In [27]:
df_ensemble_byPlaceName=pd.read_csv("./output/3-Reconstitution_URL_IREL/IREL_parlieuxreconnus.csv")


df_ensemble_byPlaceName

,placeName,Unnamed: 0
0,Abyssinie,42
1,Afrique,112380
2,Algérie,19986120
3,Allemagne,1073008
4,Amérique,58523
...,...,...
106,Toulon,54786
107,Vinh,316562
108,Wallis,42487
109,Yunnan,97978


In [28]:
liste_urlIREL = list(df_ensemble_byPlaceName["IREL_url"].str.split("http",expand=True)[2])
for i in range (len(liste_urlIREL)):
    liste_urlIREL[i]="http"+str(liste_urlIREL[i])
# "http"+liste_urlIREL[3]
liste_urlIREL

# À activer la 1er fois : 
df_ensemble_byPlaceName["IREL_url"]=liste_urlIREL

KeyError: 'IREL_url'

In [ ]:
df_ensemble_byPlaceName

In [ ]:
df_IREL

In [ ]:
df_ensemble_byPlaceName2 = pd.merge(df_ensemble_byPlaceName,df_IREL_url,how='left',on="IREL_url")
df_ensemble_byPlaceName2 = df_ensemble_byPlaceName2[["IREL_url","NAME","latitude","longitude"]]


df_ensemble_byPlaceName2

## Merge le df des longitudes/latitudes avec df_GPELOC

In [ ]:
df_GPELOC

In [29]:
df_ensemble_byPlaceName2.to_csv("./output/4-Reconstitution_CoordGéo_GPH-IREL/IREL_GPELOC")

NameError: name 'df_ensemble_byPlaceName2' is not defined

# 3. Concaténer Les long/lat de IREL, GPH et les entités qu'il reste à anoter : 

In [ ]:
df_GPH_IREL_coord=pd.concat([df_WikiData_longlat_18981914,df_ensemble_byPlaceName2])
df_GPH_IREL_coord

In [ ]:
df_GPH_IREL_GPELOC_coord = pd.merge(df_GPH_IREL_coord,df_GPELOC,how='right',on='NAME')

df_GPH_IREL_GPELOC_coord.to_csv("./output_finaux/COORD_GPH_IREL_GPELOC.csv")
df_GPH_IREL_GPELOC_coord.to_csv("./output/4-Reconstitution_CoordGéo_GPH-IREL/COORD_GPH_IREL_GPELOC.csv")

df_GPH_IREL_GPELOC_coord

In [ ]:
df_GPH_IREL_GPELOC_coord